In [4]:
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
import seaborn as sns
import sys
import sys
import re
import os.path

## data import

In [5]:
file_name =  "clean_data_2021-11-29.csv"
folder = "../clean_equity_data/"
data_tmp = pd.read_csv(folder + file_name)

In [6]:

### MOMENTUM
data_mom = data_tmp.copy()
data_mom = data_mom.loc[(data_mom['Industry'] != 'Holding Companies')]


### GESTALT
data_gestalt = data_tmp.copy()
data_gestalt = data_gestalt.loc[(data_tmp['Sector'] != 'Financials')]

### Momentum Rank

In [7]:
# RANK ON DIFFERENT METRICS 

data_mom['3m Rank'] = data_mom['Return 3m'].rank(ascending = False)
data_mom['6m Rank'] = data_mom['Return 6m'].rank(ascending = False)
data_mom['1y Rank'] = data_mom['Return 1y'].rank(ascending = False)
data_mom['EA Rank'] = data_mom['EA ret'].rank(ascending = False)
data_mom['EA Std Rank'] = data_mom['EA ret std'].rank(ascending = False)
data_mom['Volatility Rank'] = data_mom['1 Year Volatility'].rank(ascending = True)
data_mom['MAD Rank'] = data_mom['1 Year Volatility'].rank(ascending = True)

# MOMENTUM WITH WOLATILITY AND EA
data_mom['Momentum Rank'] = (data_mom['3m Rank'] + data_mom['6m Rank'] +  data_mom['1y Rank'] + data_mom['Volatility Rank'] + data_mom['EA Std Rank']).rank(ascending = True)



### Gestalt Imputation and Rank

In [8]:
# MANAGE NaNs and NEGATIVE EARNINGS  

# set nan to median, only fundamental data
columns = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'ROC', 'ROE',
            'GPA', 'Assets Turn', 'FCFROE']
for i in columns: 
    data_gestalt.loc[data_gestalt[i].isna() ,i] = data_gestalt[i].median()


    
# set negative values to max
for i in ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B']:
    data_gestalt.loc[data_gestalt[i] < 0 ,i] = data_gestalt[i].max()
    
# set nan yield & vol to 0
data_gestalt.loc[data_gestalt['Yield'].isna(),'Yield'] = 0

In [9]:
data_gestalt['3m Rank'] = data_gestalt['Return 3m'].rank(ascending = False)
data_gestalt['6m Rank'] = data_gestalt['Return 6m'].rank(ascending = False)
data_gestalt['1y Rank'] = data_gestalt['Return 1y'].rank(ascending = False)
data_gestalt['EA Rank'] = data_gestalt['EA ret'].rank(ascending = False)
data_gestalt['EA Std Rank'] = data_gestalt['EA ret std'].rank(ascending = False)


data_gestalt['Momentum Rank'] = (data_gestalt['3m Rank'] + data_gestalt['6m Rank'] + 
                                 data_gestalt['1y Rank'] + data_gestalt['EA Std Rank']).rank(ascending = True)



# Ranking where lower value is better
for i in ['P/E', 'P/B', 'P/S', 'P/FCF', 'EV/EBIT']:
    data_gestalt[i +' Rank'] = data_gestalt[i].rank()
    

for i in ['Yield', 'ROE', 'ROC', 'FCFROE', 'GPA', 'Assets Turn']:
    data_gestalt[i + ' Rank'] = (-data_gestalt[i]).rank()
    
# Composite ranks
data_gestalt['Quality Rank'] = (data_gestalt['ROE Rank'] + data_gestalt['ROC Rank'] + data_gestalt['FCFROE Rank'] + 
                        data_gestalt['GPA Rank'] + data_gestalt['Assets Turn Rank']).rank()

data_gestalt['Value Rank'] = (data_gestalt['P/E Rank'] + data_gestalt['P/B Rank'] + data_gestalt['P/S Rank'] + data_gestalt['P/FCF Rank'] + 
                      data_gestalt['EV/EBIT Rank'] + data_gestalt['Yield Rank']).rank()
 

data_gestalt['Gestalt Rank'] = ((data_gestalt['Value Rank'] + data_gestalt['Momentum Rank'] + data_gestalt['Quality Rank'])).rank()



### Data Error Check

In [10]:
#data_mom.isna().sum()
data_gestalt.isna().sum()

Unnamed: 0                      0
Börsdata ID                     0
Company                         0
Industry                        0
Volume                          0
P/FCF                           0
Assets Turn                     0
Gross profit                   11
Tot. Assets                     2
ROC                             0
Market Cap                      0
Volatility                      0
ROE                             0
FCF                             2
Total Equity                    2
Return 3m                       0
Return 6m                       0
Return 1y                       0
Country                         0
List                            0
Tick                            0
EV/EBIT                         0
Last Report                     2
Yahoo                           0
Performance - Perform. 1y.1    26
Yield                           0
P/E                             0
P/S                             0
P/B                             0
Info - Time   

### Sorting

In [11]:

#### MOMENTUM DATA FRAME
momentum_tmp = data_mom.sort_values(by=['Momentum Rank'])
#### Gestalt Data Frame
gestalt_tmp = data_gestalt.sort_values(by=['Gestalt Rank'])

In [12]:

compact = ['Company', 'List','Tick', 'Gestalt Rank', 'Quality Rank', 'Value Rank', 'Momentum Rank', 'EA ret std', '1 Year Volatility']
compact_mom = ['Company', 'List','Tick','Momentum Rank','EA ret std', '1 Year Volatility']
# numbers of stocks to select for diff strategies

stocks = 20
## MOMENTUM ##
momentum = momentum_tmp[0:stocks]
Momentum = momentum[compact_mom]


## TRIPLE SORT ##
gestalt = gestalt_tmp[0:stocks]
gestalt = gestalt[compact]

### Print Stock Selection

In [13]:
#gestalt 
th_props = [
  ('font-size', '14px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]
# Set CSS properties for td elements in dataframe
td_props = [('font-size', '12px'), ('text-align', 'center')]
#set caption props
caption_props = [('color', 'black'),('font-size', '22px'),
        ("text-align", "center"),
        ('font-weight', 'bold')]
# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props),
    dict(selector="caption", props=caption_props)
]
gestalt.index = range(1, len(gestalt)+1)
gestalt_view = (gestalt.style.apply(lambda x: ['background: lightgreen' if x.name in range(1, 10+1) 
                              else '' for i in x], axis=1).set_table_styles(styles)
                 .format({'EA ret std': "{:.1f}",'1 Year Volatility': "{:.1%}"})
                 .set_caption("Gestalt"))
gestalt_view

,Company,List,Tick,Gestalt Rank,Quality Rank,Value Rank,Momentum Rank,EA ret std,1 Year Volatility
1,Poolia,Small Cap,POOL B,1,21,29,18.5,1.3,45.0%
2,Björn Borg,Small Cap,BORG,2,25,42,14.5,3.2,36.7%
3,Ferronordic,Mid Cap,FNM,3,22,44,25,1.6,36.8%
4,BE Group,Small Cap,BEGR,4,37,19,43,-1.2,47.0%
5,Nilörngruppen,Small Cap,NIL B,5,14,34.5,59,4.2,40.7%
6,B3 Consulting,Small Cap,B3,6,27.5,71,39,2.9,40.5%
7,Transtema,Small Cap,TRANS,7,4,135,2,2.8,44.2%
8,Novotek,Small Cap,NTEK B,8,16,87.5,45,1.1,40.4%
9,Kabe,Small Cap,KABE B,9,77.5,11,63,1.9,31.4%
10,Byggmax,Mid Cap,BMAX,10,36,4,116,-1.8,37.1%


In [14]:
th_props = [
  ('font-size', '14px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]
# Set CSS properties for td elements in dataframe
td_props = [('font-size', '12px'), ('text-align', 'center')]
#set caption props
caption_props = [('color', 'black'),('font-size', '22px'),
        ("text-align", "center"),
        ('font-weight', 'bold')]
# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props),
    dict(selector="caption", props=caption_props)
]
Momentum.index = range(1,len(Momentum)+1)
Momentum_view = (Momentum.style.apply(lambda x: ['background: lightgreen' if x.name in range(1,10+1) 
                              else '' for i in x], axis=1).set_table_styles(styles).format({'EA ret std': "{:.1f}",'1 Year Volatility': "{:.1%}"})
                 .set_caption("Focused Momentum"))
Momentum_view

,Company,List,Tick,Momentum Rank,EA ret std,1 Year Volatility
1,Samhällsbyggnadsbolag B,Large Cap,SBB B,1,1.1,27.2%
2,NP3,Mid Cap,NP3,2,1.5,36.2%
3,Transtema,Small Cap,TRANS,3,2.8,44.2%
4,NOTE,Small Cap,NOTE,4,6.4,45.1%
5,Lindab,Mid Cap,LIAB,5,4.1,29.1%
6,Hanza,Small Cap,HANZA,6,4.0,46.0%
7,Sectra,Large Cap,SECT B,7,4.8,37.9%
8,Alcadon Group,First North,ALCA,8,1.3,38.9%
9,Nyfosa,Large Cap,NYF,9,1.2,27.2%
10,Thule,Large Cap,THULE,10.5,5.8,27.7%
